Step 1: Install and Imports

In [1]:
!pip install --quiet google-cloud-bigquery google-cloud-storage requests pandas

In [2]:
import os
import json
import time
import requests
import pandas as pd
from datetime import datetime, timezone

from google.cloud import bigquery
from google.cloud import storage


Step 2: Variables Declaration

In [3]:
PROJECT_ID = "qwiklabs-gcp-01-79d56652b122"

DATASET_ID = "aero_alerts"
RAW_TABLE = "airports_raw"
AIRPORTS_TABLE = "airports_large"
FORECAST_TABLE = "airport_forecasts"
ALERTS_TABLE = "airport_alerts"

BUCKET_NAME = f"{PROJECT_ID}-aero-alerts"  # if creation fails, we will switch to an existing bucket
GCS_OBJECT = "input/airports.csv"

SOURCE_GCS_URI = "gs://labs.roitraining.com/data-to-ai-workshop/airports.csv"


Step 3: Create Dataset

In [4]:
bq = bigquery.Client(project=PROJECT_ID)

dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"

bq.create_dataset(dataset, exists_ok=True)
print("Dataset created.")


Dataset created.


Step 4: Create our own GCS bucket to copy CSV into it for reliability and reuse

---



4.1 Create bucket

In [5]:
gcs = storage.Client(project=PROJECT_ID)

try:
    bucket = gcs.bucket(BUCKET_NAME)
    bucket.location = "US"
    gcs.create_bucket(bucket)
    print(f"Bucket created: {BUCKET_NAME}")
except Exception as e:
    print(f"Bucket create skipped or failed: {e}")


/tmp/ipython-input-4263483774.py:5: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = "US"


Bucket created: qwiklabs-gcp-01-79d56652b122-aero-alerts


4.2 Copy from training bucket into our bucket

In [7]:
src_bucket_name = "labs.roitraining.com"
src_object = "data-to-ai-workshop/airports.csv"

src_bucket = gcs.bucket(src_bucket_name)
src_blob = src_bucket.blob(src_object)

dst_bucket = gcs.bucket(BUCKET_NAME)
dst_blob = dst_bucket.blob(GCS_OBJECT)

dst_blob.rewrite(src_blob)
print(f"Copied to gs://{BUCKET_NAME}/{GCS_OBJECT}")


Copied to gs://qwiklabs-gcp-01-79d56652b122-aero-alerts/input/airports.csv


Step 5: Load CSV from GCS into BigQuery Raw Table

In [8]:
raw_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}"
gcs_uri = f"gs://{BUCKET_NAME}/{GCS_OBJECT}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE",
)

load_job = bq.load_table_from_uri(gcs_uri, raw_table_ref, job_config=job_config)
load_job.result()

print("Loaded airports_raw.")


Loaded airports_raw.


Step 6: Create a Clean Large-Airports Table

6.1 Inspect schema

In [9]:
table = bq.get_table(raw_table_ref)
[col.name for col in table.schema]

['id',
 'ident',
 'type',
 'name',
 'latitude_deg',
 'longitude_deg',
 'elevation_ft',
 'continent',
 'iso_country',
 'iso_region',
 'municipality',
 'scheduled_service',
 'icao_code',
 'iata_code',
 'gps_code',
 'local_code',
 'home_link',
 'wikipedia_link',
 'keywords']

6.2 Create large-airport subset

In [10]:
airports_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{AIRPORTS_TABLE}"

sql = f"""
create or replace table `{airports_table_ref}` AS
select
  coalesce(iata_code, ident, CAST(id AS STRING)) AS airport_id,
  name AS airport_name,
  type AS airport_type,
  latitude_deg AS lat,
  longitude_deg AS lon,
  municipality,
  iso_region,
  iso_country
from `{raw_table_ref}`
where lower(type) = 'large_airport'
  and latitude_deg is not null
  and longitude_deg is not null
"""
bq.query(sql).result()
print("Created airports_large.")


Created airports_large.


Step 7: Pull Forecasts from National Weather Service API for Each Large Airport

In [11]:
def nws_get_json(url: str) -> dict:
    headers = {
        "User-Agent": "AeroAlertsWorkshop/1.0 (contact: student@example.com)",
        "Accept": "application/geo+json, application/json",
    }
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    return r.json()


def build_forecast_summary(forecast_json: dict, max_periods: int = 6) -> str:
    periods = (forecast_json.get("properties", {}) or {}).get("periods", []) or []
    periods = periods[:max_periods]

    lines = []
    for p in periods:
        name = p.get("name")
        temp = p.get("temperature")
        temp_unit = p.get("temperatureUnit")
        wind = p.get("windSpeed")
        wind_dir = p.get("windDirection")
        short = p.get("shortForecast")
        detailed = p.get("detailedForecast")
        lines.append(
            f"{name}: {short}. Temp {temp}{temp_unit}. Wind {wind} {wind_dir}. Details: {detailed}"
        )
    return "\n".join(lines)


Pull and stage forecasts into a DataFrame

In [12]:
airports_df = bq.query(f"SELECT airport_id, airport_name, lat, lon FROM `{airports_table_ref}`").to_dataframe()
print(f"Large airports: {len(airports_df)}")
airports_df.head()


Large airports: 482


,airport_id,airport_name,lat,lon
0,AUH,Zayed International Airport,24.443764,54.651718
1,DWC,Al Maktoum International Airport,24.896356,55.161389
2,DXB,Dubai International Airport,25.252800,55.364399
3,SHJ,Sharjah International Airport,25.328600,55.517200
4,TIA,Tirana International Airport Mother Teresa,41.414700,19.720600


In [13]:
rows = []
run_ts = datetime.now(timezone.utc).isoformat()

for idx, row in airports_df.iterrows():
    airport_id = row["airport_id"]
    airport_name = row["airport_name"]
    lat = row["lat"]
    lon = row["lon"]

    try:
        points_url = f"https://api.weather.gov/points/{lat},{lon}"
        points_json = nws_get_json(points_url)

        forecast_url = points_json["properties"]["forecast"]
        forecast_json = nws_get_json(forecast_url)

        summary = build_forecast_summary(forecast_json, max_periods=6)

        rows.append({
            "run_ts_utc": run_ts,
            "airport_id": airport_id,
            "airport_name": airport_name,
            "lat": float(lat),
            "lon": float(lon),
            "forecast_url": forecast_url,
            "forecast_summary": summary
        })

        time.sleep(0.2)  # gentle pacing
    except Exception as e:
        rows.append({
            "run_ts_utc": run_ts,
            "airport_id": airport_id,
            "airport_name": airport_name,
            "lat": float(lat),
            "lon": float(lon),
            "forecast_url": None,
            "forecast_summary": f"ERROR: {str(e)}"
        })

forecast_df = pd.DataFrame(rows)
forecast_df.head()


,run_ts_utc,airport_id,airport_name,lat,lon,forecast_url,forecast_summary
0,2026-01-16T03:48:55.125721+00:00,AUH,Zayed International Airport,24.443764,54.651718,None,ERROR: 404 Client Error: Not Found for url: ht...
1,2026-01-16T03:48:55.125721+00:00,DWC,Al Maktoum International Airport,24.896356,55.161389,None,ERROR: 404 Client Error: Not Found for url: ht...
2,2026-01-16T03:48:55.125721+00:00,DXB,Dubai International Airport,25.252800,55.364399,None,ERROR: 404 Client Error: Not Found for url: ht...
3,2026-01-16T03:48:55.125721+00:00,SHJ,Sharjah International Airport,25.328600,55.517200,None,ERROR: 404 Client Error: Not Found for url: ht...
4,2026-01-16T03:48:55.125721+00:00,TIA,Tirana International Airport Mother Teresa,41.414700,19.720600,None,ERROR: 404 Client Error: Not Found for url: ht...


Step 8: Write Forecasts to BigQuery

In [14]:
forecast_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{FORECAST_TABLE}"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
    autodetect=True
)

bq.load_table_from_dataframe(forecast_df, forecast_table_ref, job_config=job_config).result()
print("Loaded airport_forecasts.")


Loaded airport_forecasts.


Step 9: Create Gemini Remote Model in BigQuery

In [15]:
MODEL_NAME = "gemini_model"
model_ref = f"{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}"

sql = f"""
CREATE OR REPLACE MODEL `{model_ref}`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'gemini-2.0-flash-001');
"""
bq.query(sql).result()
print("Gemini remote model created.")


Gemini remote model created.


Step 10: Generate Alerts with ML.GENERATE_TEXT and Store in BigQuery

In [21]:
alerts_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{ALERTS_TABLE}"

sql = f"""
create or replace table `{alerts_table_ref}` as
select
     run_ts_utc
    ,airport_id
    ,airport_name
    ,lat
    ,lon
    ,forecast_url
    ,forecast_summary
    ,json_value(
        ml_generate_text_result,
        '$.candidates[0].content.parts[0].text'
     ) as alert_text
from ML.GENERATE_TEXT(
    model `{model_ref}`,
    (
        select
             run_ts_utc
            ,airport_id
            ,airport_name
            ,lat
            ,lon
            ,forecast_url
            ,forecast_summary
            ,format(
                '''Create a concise operational weather alert for an airport.
Airport: %s (%s)
Write 3 lines:
1) Risk level: Low, Medium, High
2) Key hazards in plain language
3) Recommended action for airport ops

Forecast:
%s''',
                airport_name,
                airport_id,
                forecast_summary
            ) as prompt
        from `{forecast_table_ref}`
        where not starts_with(forecast_summary, 'ERROR:')
    ),
    struct(
         0.2 as temperature
        ,512 as max_output_tokens
    )
)
"""
bq.query(sql).result()
print("Created airport_alerts.")


Created airport_alerts.


Validate the alerts table

In [24]:
sql = f"""
select
     airport_id
    ,airport_name
    ,alert_text
from `{alerts_table_ref}`
limit 10
"""

bq.query(sql).to_dataframe()

,airport_id,airport_name,alert_text
0,PIT,Pittsburgh International Airport,Here's a concise operational weather alert for...
1,PDX,Portland International Airport,Here's a concise operational weather alert for...
2,STL,St. Louis Lambert International Airport,**STL Operational Weather Alert:**\n\n1. **Ri...
3,MDW,Chicago Midway International Airport,Here's a concise operational weather alert for...
4,ORD,Chicago O'Hare International Airport,**Airport Weather Alert: ORD**\n\n1. Risk Lev...
5,SNA,John Wayne Orange County International Airport,**SNA Operational Weather Alert:**\n\n1. **Ri...
6,LAS,Harry Reid International Airport,Here's a concise operational weather alert for...
7,MCO,Orlando International Airport,Here's a concise operational weather alert for...
8,CMH,John Glenn Columbus International Airport,Here's a concise operational weather alert for...
9,MCI,Kansas City International Airport,Here's a concise operational weather alert for...


**Automation**

This notebook is scheduled to run daily using Colab Enterprise scheduling to refresh weather data and regenerate Gemini-based airport alerts in BigQuery.